# using huggingface and distilbert 

In [2]:
import pandas as pd 
df = pd.read_csv('dataset/spamdetection.csv',sep='\t',names=['label','message']).dropna()
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
df.shape # 5574 rows and 2 columns

(1782, 2)

In [4]:
x = list(df['message'])
y = list(df['label'])

In [5]:
y = list(pd.get_dummies(y,drop_first=True) ['spam']) #take spam column then convert to list

In [6]:
y

[0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


#  train test split

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [8]:
from transformers import DistilBertTokenizerFast # differnt architecture use diffenet tokenizier 
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased') # load from this model

In [9]:
#row must be nullfree that means dropna() must be used when load dataset

train_encoding = tokenizer(X_train,truncation=True, padding=True) # padding keep sentence same length 
test_encoding = tokenizer(X_test,truncation=True, padding=True)  

# Convert this encoding into dataset objects

In [10]:
import tensorflow as tf # as we used tensorflow so we need to use tensor

In [11]:
train_dataset = tf.data.Dataset.from_tensors((dict(train_encoding),y_train)) #data object
train_dataset

<TensorDataset shapes: ({input_ids: (1425, 238), attention_mask: (1425, 238)}, (1425,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>

In [24]:
test_dataset = tf.data.Dataset.from_tensors((dict(test_encoding), y_test)) #data object
test_dataset

<TensorDataset shapes: ({input_ids: (357, 109), attention_mask: (357, 109)}, (357,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>

In [13]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments 
#1 is used for sentiment analysis, 2,3 training purpose

In [17]:
training_args = TFTrainingArguments(
output_dir='.results',
num_train_epochs = 10,
per_device_train_batch_size =8,
per_device_eval_batch_size =16,
warmup_steps = 500,
weight_decay = 0.1,
logging_dir ='./logs',
logging_steps = 10)

In [19]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
trainer = TFTrainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=test_dataset)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

ValueError: in user code:

    c:\users\mehedi\jupyter notebook projects\transformerfornlp\transvenv\lib\site-packages\transformers\trainer_tf.py:706 distributed_training_steps  *
        self.args.strategy.run(self.apply_gradients, inputs)
    c:\users\mehedi\jupyter notebook projects\transformerfornlp\transvenv\lib\site-packages\transformers\trainer_tf.py:648 apply_gradients  *
        gradients = self.training_step(features, labels, nb_instances_in_global_batch)
    c:\users\mehedi\jupyter notebook projects\transformerfornlp\transvenv\lib\site-packages\transformers\trainer_tf.py:631 training_step  *
        per_example_loss, _ = self.run_model(features, labels, True)
    c:\users\mehedi\jupyter notebook projects\transformerfornlp\transvenv\lib\site-packages\transformers\trainer_tf.py:753 run_model  *
        outputs = self.model(features, labels=labels, training=training)[:2]
    c:\users\mehedi\jupyter notebook projects\transformerfornlp\transvenv\lib\site-packages\transformers\mod